In [1]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

# https://dvr.capitol.texas.gov/
level    = 'cntyvtd'
proposal = 'plans2168'  # for 2010 enacted, use 'planc2100' for US congress, 'plans2100' for TX Senate, or 'planh2100' for TX House
contract = 0
# contract = 'proposal'

# D = Data()
S = Space(level=level, proposal=proposal, contract=contract)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'],
         max_steps   = 5,
         random_seed = 13,
         defect_cap  = 100,
         pop_deviation_target=10)
# M.run_chain()
# M.post_process()

/home/jupyter/MathVGerrmandering_CMAT_2021
getting states
Get proposal    ... using existing table ... success!
Get nodes       ... using existing table ... success!
Get graph       ... processing ... using existing graph ... success!


In [2]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output}"
plans = run_query(query).pivot(index='geoid', columns='plan', values='district')

node_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(node_cols)} from {M.nodes}"
nodes = run_query(query).set_index('geoid')

Loading BokehJS ...

In [3]:
repl = {c:str(c) for c in plans.columns}
plans.rename(columns=repl, inplace=True)

geo = gpd.GeoSeries.from_wkt(nodes['polygon'], crs=crs_census).simplify(0.003).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
gdf = nodes.drop(columns='polygon').join(plans, how='inner')
gdf = gpd.GeoDataFrame(gdf, geometry=geo).reset_index()
gdf.head(3)

,geoid,county,total_pop,aland,density,polsby_popper,0,1,2,3,4,5,geometry
0,025000010,Bee,2838,1.083924,2618.265936,30.50,27,27,27,27,27,27,"POLYGON ((-97.73855 28.38500, -97.74647 28.393..."
1,025000013,Bee,2097,144.627052,14.499362,47.13,27,27,27,27,27,27,"POLYGON ((-97.90409 28.51056, -97.88110 28.518..."
2,025000003,Bee,10625,19.599254,542.112468,35.01,27,27,27,27,27,27,"POLYGON ((-97.73471 28.38578, -97.72887 28.386..."


In [ ]:
width = 600
xlim = [-106.2, -94.0]
ylim = [ 25.4 ,  36.6]
aspect = (ylim[1] - ylim[0]) / (xlim[1] - xlim[0])
height = round(aspect * width)

def f(cmap):
    fn = str(M.gpickle)
    fn = f'{fn[:fn.rfind("_")]}_{M.random_seed}_animation_{cmap}.html'
    print(f'cmap={cmap} ... saving animation to {fn}')
    fig = gdf.plot_bokeh(
        colormap = cmap,
        hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
        fill_alpha = 0.5,
        line_alpha = 0.05,
        return_html = True,
        show_figure = True,
        slider = list(repl.values()),
        slider_name = "PLAN",
        show_colorbar = False,
        xlim = xlim,
        ylim = ylim,
        figsize = (width, height),
    )
    with open(fn, 'w') as file:
        file.write(fig)
    return fig

f('Turbo')

# for cmap in ['Accent', 'Blues', 'Bokeh', 'BrBG', 'BuGn', 'BuPu', 'Category10', 'Category20', 'Category20b', 'Category20c',
#              'Cividis', 'Colorblind', 'Dark2', 'GnBu', 'Greens', 'Greys', 'Inferno', 'Magma', 'OrRd', 'Oranges', 'PRGn',
#              'Paired', 'Pastel1', 'Pastel2', 'PiYG', 'Plasma', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy',
#              'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Set1', 'Set2', 'Set3', 'Spectral', 'Turbo', 'Viridis', 'YlGn', 'YlGnBu',
#              'YlOrBr', 'YlOrRd']:
#     f(cmap)